# **Project 2:**

In [57]:
import numpy as np
import pandas as pd
from src.Ward import Ward, initialize_wards
from src.Patient import Patient, initialize_patients

## initialization

In [58]:
F_bed_capacity = 30 ##### Number to optimize #####

#patient type from 'A' to 'F'
patient_types = ['A', 'B', 'C', 'D', 'E', 'F']
bed_capacity = np.array([55,40,30,20,20, F_bed_capacity])
arrivals_pr_day = np.array([14.5,11.0,8.0,6.5,5.0, 13.0])
mean_length_of_stay = np.array([2.9,4.0,4.5,1.4,3.9, 2.0])
urgency_points = np.array([7,5,2,10,5, 0])

#reloaction probability
relocation_probability = np.array([
    [0.0, 0.05, 0.10, 0.05, 0.80, 0.00],
    [0.2, 0, 0.50, 0.15, 0.15, 0.00],
    [0.30, 0.20, 0, 0.20, 0.30, 0.00],
    [0.35, 0.30, 0.05, 0, 0.3, 0.00],
    [0.20, 0.10, 0.60 ,0.10, 0, 0.00],
    [0.20, 0.20, 0.20, 0.20, 0.20 ,0]
    ])

#Dataframe containing all the data (indexed by patient type)
df = pd.DataFrame({
    'Patient Type': patient_types,
    'Bed Capacity': bed_capacity,
    'Arrivals per day': arrivals_pr_day,
    'Mean Length of Stay': mean_length_of_stay,
    'Urgency Points': urgency_points
    }).set_index('Patient Type')
df

,Bed Capacity,Arrivals per day,Mean Length of Stay,Urgency Points
Patient Type,,,,
A,55,14.5,2.9,7
B,40,11.0,4.0,5
C,30,8.0,4.5,2
D,20,6.5,1.4,10
E,20,5.0,3.9,5
F,30,13.0,2.0,0


In [59]:
wards = initialize_wards(df) #list of wards
wards

[A Ward with 55 beds and 0 patients currently admitted.,
 B Ward with 40 beds and 0 patients currently admitted.,
 C Ward with 30 beds and 0 patients currently admitted.,
 D Ward with 20 beds and 0 patients currently admitted.,
 E Ward with 20 beds and 0 patients currently admitted.,
 F Ward with 30 beds and 0 patients currently admitted.]

In [60]:
arrival_interval_function = lambda type: np.random.exponential(1/df["Arrivals per day"][type])
occupancy_time_function = lambda type: np.random.exponential(df["Mean Length of Stay"][type])

In [61]:
total_time = 100 #total days to permit admissions
patients = initialize_patients(total_time, patient_types, arrival_interval_function, occupancy_time_function)
patients

[Patient B with Admission at time 0.03633993153539427,
 Patient F with Admission at time 0.03694482844727043,
 Patient E with Admission at time 0.05042416654097004,
 Patient F with Admission at time 0.05388217634877542,
 Patient A with Admission at time 0.056206166239967485,
 Patient F with Admission at time 0.07434772559307694,
 Patient E with Admission at time 0.07646768180861752,
 Patient F with Admission at time 0.10064884870034856,
 Patient E with Admission at time 0.10360489684308671,
 Patient A with Admission at time 0.11735044986076376,
 Patient A with Admission at time 0.1279114656529852,
 Patient C with Admission at time 0.1771023546427819,
 Patient A with Admission at time 0.19629683274191279,
 Patient A with Admission at time 0.22712450389021688,
 Patient B with Admission at time 0.2369219991306908,
 Patient C with Admission at time 0.25314589847821684,
 Patient F with Admission at time 0.260085472155861,
 Patient C with Admission at time 0.2857355503802087,
 Patient B with

In [62]:
def simulation_loop(patients, wards, relocation_probabilities):
    """Runs the simulation loop

    Args:
        patients (list): list of patient objects
        wards (list): list of ward objects
        relocation_probabilities (np.array): probability matrix for patient relocation

    Returns:
        urgency_points, total_lost: total urgency points, and total lost during the simulation
    """
    urgency_points = 0
    total_lost = 0
    while len(patients) > 0:
        patient = patients.pop(0)
        print(patient)
        if patient.next_event == "Admission":
            urgency_points += patient.occupy_bed(wards, patients, relocation_probabilities)
            total_lost += patient.lost
        else:
            patient.discharge(wards)
    return urgency_points, total_lost

In [63]:
urgency_points, total_lost = simulation_loop(patients, wards, relocation_probability)

Patient B with Admission at time 0.03633993153539427
Patient F with Admission at time 0.03694482844727043
Patient E with Admission at time 0.05042416654097004
Patient F with Admission at time 0.05388217634877542
Patient A with Admission at time 0.056206166239967485
Patient F with Admission at time 0.07434772559307694
Patient E with Admission at time 0.07646768180861752
Patient F with Admission at time 0.10064884870034856
Patient E with Admission at time 0.10360489684308671
Patient E with Discharge at time 0.11209247550583072
Patient A with Admission at time 0.11735044986076376
Patient A with Admission at time 0.1279114656529852
Patient C with Admission at time 0.1771023546427819
Patient A with Admission at time 0.19629683274191279
Patient F with Discharge at time 0.22072106190537313
Patient A with Admission at time 0.22712450389021688
Patient B with Admission at time 0.2369219991306908
Patient C with Admission at time 0.25314589847821684
Patient F with Admission at time 0.2600854721558

In [64]:
print("Urgency points: ", urgency_points)
print("Total lost: ", total_lost)

Urgency points:  3065
Total lost:  211
